### ETL back data from RedAlert xlsx export

In [1]:
# Python imports
import os

# Django imports
from django.utils import timezone
import datetime as dt

# 3rd party imports
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_rows', 500)

# Intraproject imports
import django_init
from django.conf import settings
from calls.models import Incident, DISP

In [2]:
# load into pandas dataframe with proper cleaning & transforming
input_path = os.path.join(
    settings.BASE_DIR.parent, 
    'data', '2025_01_14_hfd_incident_log.xls',
)

df = pd.read_excel(input_path, parse_dates=[['Date', 'Alarm']])
df = df.dropna(subset=['Incident#'])
df['FDID'] = df['FDID'].astype(str)
df['Date_Alarm'] = df['Date_Alarm'].astype('datetime64[s]')
broken_type = 'Unintentional system/detector operation (no '
df.loc[df['Type']==broken_type, 'Type'] = broken_type + 'fire)'

df.head(4)

,Date_Alarm,FDID,Incident#,Num,Address,Suite,Zip,Type,Lgth
0,2000-01-02 13:41:00,14013.0,2000-000001,10,BARBARA LA.,,12590,Natural vegetation fire,0.0
1,2000-01-04 12:30:00,14013.0,2000-000002,2015,ROUTE 9 - UNIT 16,,12590,Emergency medical service (EMS) Incident,0.5
2,2000-01-04 12:36:00,14013.0,2000-000003,52,OSBORNE HILL RD.,,12590,Emergency medical service (EMS) Incident,0.7
3,2000-01-05 22:13:00,14013.0,2000-000004,NaN,NEW HAMBURG RD. @ WHEELER,,12590,"Service call, other",1.0


In [3]:
# load into database

for index, record in tqdm(df.iterrows()):
    Incident.objects.get_or_create(
        num = str(record['Incident#']).strip(),
        defaults = dict(
            fd_id = str(record['FDID']).strip('.0'),
            dtg_alarm = record['Date_Alarm'].to_pydatetime().astimezone(settings.TIME_ZONE_OBJ),
            street_number = str(record['Num']).strip() if pd.notna(record['Num']) else None,
            route = str(record['Address']).strip() if pd.notna(record['Address']) else None,
            suite = str(record['Suite']).strip() if pd.notna(record['Suite']) else None,
            postal_code = str(record['Zip']).replace('-', '').strip()[:5] if pd.notna(record['Zip']) else None,
            disp = DISP.objects.get_or_create(type_str=str(record['Type']).strip())[0] if pd.notna(record['Type']) else None,
            duration = record['Lgth'] if pd.notna(record['Lgth']) else None,
        )
    )

15483it [00:24, 629.79it/s]
